推奨：

/content/drive/MyDrive/ArknightsGameData_autoWiki3/

上にフォルダ作って実装済み（対象ではない）charIDの1列のcsvを用意しておくと今後加筆していく

「zh_CN_IDList.csv」
「ja_JP_IDList.csv」

wiki編集者素材のデータも[name,htmlソースコード]で「編集者素材.csv」をいれておくと楽

MyDrive直下に「ArknightsGameData_autoWiki3」ってフォルダ作ってファイルをぶちこめ

これはshidanoが使ってるファイル群
https://drive.google.com/drive/folders/1-H2eyUCFJpvkf_cwnOJrIgbNb4ifPLHX?usp=sharing

## 初期セットアップ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import pprint
import html
import pandas as pd
import re
import os
from pathlib import Path
from itertools import chain
import datetime
from pytz import timezone
import csv
from difflib import HtmlDiff


In [ ]:
!git clone https://github.com/Kengxxiao/ArknightsGameData.git

Cloning into 'ArknightsGameData'...
remote: Enumerating objects: 44213, done.
remote: Counting objects: 100% (10134/10134), done.
remote: Compressing objects: 100% (3467/3467), done.
remote: Total 44213 (delta 6766), reused 9668 (delta 6631), pack-reused 34079
Receiving objects: 100% (44213/44213), 396.23 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (25201/25201), done.
Updating files: 100% (16420/16420), done.


In [ ]:
def half2full(s: str):
  if(s==None):
    return s
  ss = s
  ss = ss.replace('+','＋')
  ss = ss.replace('-','－')
  ss = ss.replace('%','％')
  return ss

def kantan_honyaku(s: str):
  #do not use skiruuuuuuuuuu
  if(s==None):
    return s
  ss = s
  ss = ss.replace("部署费用","配置コスト")
  ss = ss.replace("再部署时间","再配置時間")
  ss = ss.replace("第一天赋效果增强","第一素質強化")
  ss = ss.replace("第二天赋效果增强","第二素質強化")
  ss = ss.replace("天赋效果增强","素質強化")
  ss = ss.replace("攻击速度","攻撃速度")
  ss = ss.replace("生命上限","最大HP")
  ss = ss.replace("攻击力","攻撃力")
  ss = ss.replace('防御力','防御力')
  ss = ss.replace('法术抗性','術耐性')
  ss = ss.replace('，','、')
  ss = ss.replace('攻击范围','攻撃範囲')
  ss = ss.replace('攻击间隔','攻撃間隔')
  ss = ss.replace('攻击装有','合計')
  ss = ss.replace('发弹药，打完后结束','発の銃弾を撃ち切るとスキルが終了')
  ss = ss.replace('可随时停止技能','手動でスキルを停止可能')

  return ss

def senzaiT(s: str):
  return half2full(kantan_honyaku(s))

def getTeam(id):
  if(id==None):
    return "None"
  if(handbook_team_table[id]==None):
    return id
  return handbook_team_table[id]["powerName"]

In [ ]:
def getItemName(itemID):
  if(item_table["items"][itemID]==None):
    return cn_item_table["items"][itemID]
  return item_table["items"][itemID]["name"]

def getCharName(charID):
  return character_table[char_id]["name"]

def getWiki3Contents(name):
  #print(name)
  if(name in checkedWiki3Contents):
    return checkedWiki3Contents[name]
  if(wiki編集者素材.find(name)==-1):
    return ("["+name+"]が見つかりませんでした")
  pattern=r'.*<p>(<a href="https://wiki3.jp/arknightsjp/page/.*?"><img src.*?'+name+'</a>)</p>'
  data=re.search(pattern,wiki編集者素材)[1]
  if(data==None):
    return ("["+name+"]が見つかりませんでした")
  checkedWiki3Contents[name]=data
  print(checkedWiki3Contents[name])
  with open('/content/drive/MyDrive/ArknightsGameData_autoWiki3/編集者素材.csv', 'a') as f:
    print(name+','+checkedWiki3Contents[name]+'\n', file=f)
  return checkedWiki3Contents[name]

In [ ]:
def senzai(s1,s5,how):
  index=0;
  check= ["0","1","2","3","4","5","6","7","8","9",".","%"]
  for up in re.finditer(r'(&lt;@ba\.talpu&gt;|<@ba\.talpu>)(\(|（)(.*?)(\)|）)(&lt;/&gt;|</>)',s5):
    upgrade="（潜在"+str(how)+"で"+up.group(3)+"）"
    targetindex=up.start(0)-index
    while True:
      if(targetindex>=len(s1)):
        break
      if(s1[targetindex] in check):
        targetindex+=1
        index+=1
      else:
        break
    s1=s1[:targetindex]+upgrade+s1[targetindex:]
    index+=len(up.group(0))-len(upgrade)
  return s1


# 設定変更とデータ入力

In [ ]:
#初期化と初期設定、ここかこの下を実行せよ
# "ja_JP" or "zh_CN"

lang = "zh_CN"
add = []

In [ ]:
lang = "ja_JP"
add = []

In [ ]:
#標準入力からwiki編集者素材ページのソースを編集モードでコピーして入力してください
wiki編集者素材 = input()

a


In [ ]:
##初期化後に実行が必要
checkedWiki3Contents={}

if not os.path.exists('/content/drive/MyDrive/ArknightsGameData_autoWiki3'):
  os.mkdir('/content/drive/MyDrive/ArknightsGameData_autoWiki3')
%cd /content/drive/MyDrive/ArknightsGameData_autoWiki3
if not os.path.exists(f'/content/drive/MyDrive/ArknightsGameData_autoWiki3/{lang}_IDList.csv'):
  Path(lang+"_IDList.csv").touch()
  old = []
  old = pd.DataFrame({'id':old})
  old.to_csv(f'/content/drive/MyDrive/ArknightsGameData_autoWiki3/{lang}_IDList.csv',columns=['id'],index=False);
if not os.path.exists('/content/drive/MyDrive/ArknightsGameData_autoWiki3/編集者素材.csv'):
  Path("編集者素材.csv").touch()
  sozai = []
  sozai = pd.DataFrame({'name':'data'})
  sozai.to_csv(f'/content/drive/MyDrive/ArknightsGameData_autoWiki3/編集者素材.csv',columns=['id'],index=False);

%cd /content/ArknightsGameData
with open(f"{lang}/gamedata/excel/character_table.json") as f:
    character_table = json.load(f)

with open(f"{lang}/gamedata/excel/skill_table.json") as f:
    skill_table = json.load(f)

with open(f"{lang}/gamedata/excel/building_data.json") as f:
    building_data = json.load(f)

with open(f"{lang}/gamedata/excel/uniequip_table.json") as f:
    uniequip_table = json.load(f)

with open("ja_JP/gamedata/excel/item_table.json") as f:
    item_table = json.load(f)

with open("zh_CN/gamedata/excel/item_table.json") as f:
    cn_item_table = json.load(f)

with open("ja_JP/gamedata/excel/handbook_team_table.json") as f:
    handbook_team_table = json.load(f)

with open('/content/drive/MyDrive/ArknightsGameData_autoWiki3/編集者素材.csv', mode='r') as f:
    checkedWiki3Contents = {rows[0]:rows[1] for rows in csv.reader(f)}

with open(f"{lang}/gamedata/excel/battle_equip_table.json") as f:
    battle_equip_table = json.load(f)

old = pd.read_csv(f'/content/drive/MyDrive/ArknightsGameData_autoWiki3/{lang}_IDList.csv')
old = old.values.tolist()
old=list(chain.from_iterable(old))
for char_id in character_table:
  if(char_id not in old):
    old.append(char_id)
    add.append(char_id)
    print(char_id)
old = pd.DataFrame({'id':old})
old.to_csv(f'/content/drive/MyDrive/ArknightsGameData_autoWiki3/{lang}_IDList.csv',columns=['id'],index=False);


/content/drive/MyDrive/ArknightsGameData_autoWiki3
/content/ArknightsGameData
char_341_sntlla
char_2012_typhon


# コア・キャラクター単位処理

In [ ]:
def charTable(char_id):
  # 素質
  output=""
  cntext=""
  if(character_table[char_id]["talents"]==None):
    return ""

  if(lang=="zh_CN" and not character_table[char_id]["tagList"]==None):
    rID=[]
    cntext += f'<p>nationId：{getTeam(character_table[char_id]["nationId"])}</p><p>groupId：{getTeam(character_table[char_id]["groupId"])}</p><p>teamId：{getTeam(character_table[char_id]["teamId"])}</p>'
    cntext += "<p>募集タグ：【"+"】【".join(character_table[char_id]["tagList"])+"】</p><p>コスト："
    for ev in character_table[char_id]["phases"]:
      cntext +=str(ev["attributesKeyFrames"][0]["data"]["cost"])+"&nbsp;;&nbsp;"
      rID.append(ev["rangeId"])
    rID.append("")
    rID.append("")
    rID.append("")
    cntext+="</p><p>再配置時間："+str(character_table[char_id]["phases"][0]["attributesKeyFrames"][0]["data"]["respawnTime"])+"s</p><p>&nbsp;</p>"

    cntext+=  '<div class="uk-overflow-container"><table class="uk-table"><thead><tr><th scope="row">&nbsp;</th><th scope="col">初期</th><th scope="col">初期MAX</th><th scope="col">昇進1MAX</th><th scope="col">昇進2MAX</th><th scope="col">信頼度</th></tr></thead><tbody>'
    data=[
        ["最大HP","","","","",""],
        ["攻撃力","","","","",""],
        ["防御力","","","","",""],
        ["術耐性","","","","",""],
    ]
    stmem=["maxHp","atk","def","magicResistance"]
    for i in range(4):
      if(character_table[char_id]["phases"][0]["attributesKeyFrames"][0]==None):
        data[i][1]="なし"
      else:
        data[i][1]=str(character_table[char_id]["phases"][0]["attributesKeyFrames"][0]["data"][stmem[i]])

      for j in range(3):
        if(j>=len(character_table[char_id]["phases"])):
          data[i][j+2]="なし"
        else:
          data[i][j+2]=str(character_table[char_id]["phases"][j]["attributesKeyFrames"][1]["data"][stmem[i]])

    for i in range(4):
      if(int(character_table[char_id]["favorKeyFrames"][1]["data"][stmem[i]])==0 or character_table[char_id]["favorKeyFrames"][1]["data"][stmem[i]]==None):
        data[i][5]=""
      else:
        data[i][5]="＋"+str(character_table[char_id]["favorKeyFrames"][1]["data"][stmem[i]])
      cntext+="<tr><th>"+data[i][0]+"</th>"
      for j in range(4):
        cntext+='<td class="uk-text-right">'+data[i][j+1]+'</td>'
      cntext+='<td class=>'+data[i][5]+'</td></tr>'
    cntext+=f'<tr><th scope="row">攻撃範囲</th><td colspan="2" rowspan="1">{rID[0]}</td><td>{rID[1]}</td><td>{rID[2]}</td><td>&nbsp;</td></tr></tbody></table></div><h3>潜在</h3><div class="uk-overflow-container"><table class="uk-table">'
    ptc=2
    for pot in character_table[char_id]["potentialRanks"]:
      cntext+=f'<tr><th>{ptc}</th><td>{senzaiT(pot["description"])}</td></tr>'
      ptc+=1
    cntext+='</tbody></table></div><p>&nbsp;</p>'

    ptc=1
    for ev in character_table[char_id]["phases"]:
      if(ev["evolveCost"]==None):
        continue
      cntext+="<p>昇進"+str(ptc)+"</p>"
      for item in ev["evolveCost"]:
        cntext+="<p>" + getWiki3Contents(getItemName(item["id"])) +"&nbsp;x"+str(item["count"])+ "</p>"
      ptc+=1
      cntext+="<p>&nbsp;</p>"

  output+=cntext+"<p>&nbsp;</p>"

  l = []
  output+='<div class="uk-overflow-container"><table class="uk-table"><thead><tr><th scope="row">素質</th><th scope="col">習得条件</th><th scope="col">効果</th></tr></thead><tbody>'

  if(character_table[char_id]["rarity"]==0 or character_table[char_id]["rarity"]=="TIER_1"):
     for i in range(len(character_table[char_id]["talents"])):
      if(character_table[char_id]["talents"][i]["candidates"]==None):
        continue
      for j in range(len(character_table[char_id]["talents"][i]["candidates"])):
          name = character_table[char_id]["talents"][i]["candidates"][j]["name"]
          requiredPotentialRank = character_table[char_id]["talents"][i]["candidates"][j]["requiredPotentialRank"]+1
          description = character_table[char_id]["talents"][i]["candidates"][j]["description"]
          if(requiredPotentialRank==1):
            requiredPotentialRank="初期"
          else:
            requiredPotentialRank="潜在"+str(requiredPotentialRank)
          output+=f'<tr><th>{name}</th><td>{requiredPotentialRank}</td><td>{half2full(description)}</td></tr>'
  else:
    data=[[[],[["",""],["",""],["",""]]],[[],[["",""],["",""],["",""]]]]
    for i in range(len(character_table[char_id]["talents"])):
      if(character_table[char_id]["talents"][i]["candidates"]==None):
        continue
      for j in range(len(character_table[char_id]["talents"][i]["candidates"])):
          name = character_table[char_id]["talents"][i]["candidates"][j]["name"]
          unlockCondition_pase = str(character_table[char_id]["talents"][i]["candidates"][j]["unlockCondition"]["phase"])
          unlockCondition_pase = int(unlockCondition_pase[-1:])
          unlockCondition_lv = character_table[char_id]["talents"][i]["candidates"][j]["unlockCondition"]["level"]
          requiredPotentialRank = character_table[char_id]["talents"][i]["candidates"][j]["requiredPotentialRank"]+1
          description = character_table[char_id]["talents"][i]["candidates"][j]["description"]
          if(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='1+'):
            daiX=0
          elif(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='2+'):
            daiX=1
          elif(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='#'):
            daix=0
          else:
            daiX=int(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"])-1
          if(data[daiX][1][unlockCondition_pase][0]==''):
            if(unlockCondition_pase==0 and unlockCondition_lv==1):
              data[daiX][1][unlockCondition_pase] = ["初期", description ]
            elif(unlockCondition_pase==0):
              data[daiX][1][unlockCondition_pase] = ["Lv"+str(unlockCondition_lv) , description ]
            elif(unlockCondition_lv==1):
              data[daiX][1][unlockCondition_pase] = ["昇進"+str(unlockCondition_pase), description ]
            else:
              data[daiX][1][unlockCondition_pase] = ["昇進"+str(unlockCondition_pase)+"Lv"+str(unlockCondition_lv) , description ]
          else:
            data[daiX][1][unlockCondition_pase][1] = senzai(data[daiX][1][unlockCondition_pase][1],description,requiredPotentialRank)
          if(name not in data[daiX][0]):
            data[daiX][0].append(name)

    for i in range(2):
      "</p><p>&darr;</p><p>".join(data[daiX][0])
    for tar in data:
      text=[]
      # add 0607 popi
      if tar == [[], [['', ''], ['', ''], ['', '']]]:
          continue

      for t in tar[1]:
          if t[0] == "":
              continue
          text.append(f"<td>{t[0]}</td><td>{t[1]}</td>")
      text = kantan_honyaku("</tr><tr>".join(text))

      output += f'<tr><th rowspan="{sum(x[0]>"" for x in tar[1])}" scope="row">{str(tar[0][0])}</th>{text}</tr>'

  output+='</tbody></table></div>'
  output = "<h1>"+character_table[char_id]["name"]+"</h1>\n<h3 id=\"content_header_main_content_h3_5\">素質</h3>\n" + output

  for skill in character_table[char_id]["skills"]:
    output += skillTable(skill["skillId"])+"<p>&nbsp;</p>"
  if(lang=="zh_CN"):
    output += skillUpgrade(char_id)+"<p>&nbsp;</p>"
  output += building_skill(char_id)+"<p>&nbsp;</p>"

  return output


### スキル

In [ ]:
def skillTable(skill_id):
  # スキル
  # skill_id = "skchr_texas2_1"


  l = []
  if(skill_id==None):
      return ""
  for i in range(len(skill_table[skill_id]["levels"])):
      blackboard = skill_table[skill_id]["levels"][i]["blackboard"]
      level = i + 1
      description = skill_table[skill_id]["levels"][i]["description"]
      initSp = skill_table[skill_id]["levels"][i]["spData"]["initSp"]
      spCost = skill_table[skill_id]["levels"][i]["spData"]["spCost"]
      duration = skill_table[skill_id]["levels"][i]["duration"]

      for d in blackboard:
        for t in re.findall("{.+?}", description):
            t = t[1:-1].split(":")
            if len(t) == 1:
                if t[0] == d["key"]:
                    description = description.replace("{" + d["key"] + "}", str(int(d["value"])))
                elif t[0] == f"-{d['key']}":
                    description = description.replace("-{" + t[0] + "}", str(int(d["value"])))
            else:
                description = description.replace("{" + d["key"] + ":0%}", str(int(d["value"]*100)) + "%")

      description = kantan_honyaku(description)
      if(int(duration)>0):
        duration=f"{int(duration)}秒"
      else:
        duration="-"
      l.append([level, description, initSp, spCost, duration])

  df = pd.DataFrame(l, columns=["Lv", "効果", "初期", "消費", "持続"])

  html_table = df.to_html(index=False)
  html_table = html_table.replace("&lt;@ba.vup&gt;", "<span class=\"tc_yellow\">").replace("&lt;/&gt;", "</span>")

  html_table = html_table.replace("<table border=\"1\" class=\"dataframe\">", "<table class=\"uk-table\">")
  html_table = html_table.replace("<tr style=\"text-align: right;\">", "<tr>")

  html_table = html_table.replace(
    """\n  <thead>
    <tr>
      <th>Lv</th>
      <th>効果</th>
      <th>初期</th>
      <th>消費</th>
      <th>持続</th>
    </tr>
  </thead>
  <tbody>""",
    """\n  <thead>
  </thead>
  <tbody>
    <tr>
    <th scope="row" style="white-space: nowrap; width: 0%;">Lv</th>
    <th>効果</th>
    <th style="white-space: nowrap; width: 0%;">初期</th>
    <th style="white-space: nowrap; width: 0%;">消費</th>
    <th style="white-space: nowrap; width: 0%;">持続</th>
    </tr>""")

  html_table = re.sub("""<tr>
      <td>(.*?)</td>
      <td>(.*?)</td>
      <td>(.*?)</td>
      <td>(.*?)</td>
      <td>(.*?)</td>""",
      """<tr>
      <th>\\1</th>
      <td>\\2</td>
      <td class="uk-text-right">\\3</td>
      <td class="uk-text-right">\\4</td>
      <td class="uk-text-right" style="white-space:nowrap">\\5</td>""",
        html_table)

  html_table = re.sub("&lt;\$ba\..*&gt;(.*?)</span>",
        "<span class=\"uk-text-bold\"><u>\\1</u></span>",
        html_table)

  spType_skillType2ja = {
      "1,1": "&nbsp;[自動回復]&nbsp;[手動発動]",
      "1,2": "&nbsp;[自動回復]&nbsp;[自動発動]",
      "2,1": "&nbsp;[攻撃回復]&nbsp;[手動発動]",
      "2,2": "&nbsp;[攻撃回復]&nbsp;[自動発動]",
      "4,1": "&nbsp;[被撃回復]&nbsp;[手動発動]",
      "4,2": "&nbsp;[被撃回復]&nbsp;[自動発動]",
      "8,0": "&nbsp;[パッシブ]",
      "INCREASE_WITH_TIME,MANUAL": "&nbsp;[自動回復]&nbsp;[手動発動]",
      "INCREASE_WITH_TIME,AUTO": "&nbsp;[自動回復]&nbsp;[自動発動]",
      "INCREASE_WHEN_ATTACK,MANUAL": "&nbsp;[攻撃回復]&nbsp;[手動発動]",
      "INCREASE_WHEN_ATTACK,AUTO": "&nbsp;[攻撃回復]&nbsp;[自動発動]",
      "INCREASE_WHEN_TAKEN_DAMAGE,MANUAL": "&nbsp;[被撃回復]&nbsp;[手動発動]",
      "INCREASE_WHEN_TAKEN_DAMAGE,AUTO": "&nbsp;[被撃回復]&nbsp;[自動発動]",
      "8,PASSIVE": "&nbsp;[パッシブ]",
  }

  skillType = str(skill_table[skill_id]["levels"][0]["skillType"])
  spType = str(skill_table[skill_id]["levels"][0]["spData"]["spType"])
  k = f"{spType},{skillType}"

  output = "<div class=\"uk-overflow-container\">\n" + html_table + "\n</div>\n<p>&nbsp;</p>"
  output = "<p>"+skill_table[skill_id]["levels"][0]["name"]+spType_skillType2ja[k]+"</p>\n"+output+"\n"
  return output


In [ ]:
def skillUpgrade(char_id):
  output='<h3>スキル強化</h3><div class="uk-overflow-container"><table class="uk-table"><thead><tr><th scope="row">スキルレベル</th><th scope="col">必要素材</th></tr></thead><tbody>'
  before=1
  after=2
  for lv in character_table[char_id]["allSkillLvlup"]:
    output+="<tr><th>" + str(before) + "&rarr;" + str(after) +"</th><td>"
    for item in lv["lvlUpCost"]:
      output+="<p>" + getWiki3Contents(getItemName(item["id"])) +"&nbsp;x"+str(item["count"])+ "</p>"
    before+=1
    after+=1
    output+="</td></tr>"

  skillnum=1
  for skill in character_table[char_id]["skills"]:
    before=7
    after=8

    for lv in skill["levelUpCostCond"]:
      output+="<tr><th><p>&nbsp;</p><p>S" + str(skillnum) + "</p><p>" + str(before) + "&rarr;" + str(after) +"</p></th><td>"
      for item in lv["levelUpCost"]:
        output+="<p>" + getWiki3Contents(getItemName(item["id"])) +"&nbsp;x"+str(item["count"])+ "</p>"
      before+=1
      after+=1
    skillnum+=1
    output+="</td></tr>"

  output+="</tbody></table></div>\n"
  return output

### 基地スキル

In [ ]:
# 基地スキル
def building_skill(char_id):
  if(char_id not in building_data["chars"].keys()):
    return ""

  room_dict = {
      "MANUFACTURE": "製造所",
      "TRADING": "貿易所",
      "POWER": "発電所",
      "WORKSHOP": "加工所",
      "CONTROL": "制御中枢",
      "MEETING": "応接室",
      "DORMITORY": "宿舎",
      "HIRE": "事務室",
      "TRAINING": "訓練室"
  }

  l = []
  for i in range(len(building_data["chars"][char_id]["buffChar"])):
      buffData = building_data["chars"][char_id]["buffChar"][i]["buffData"]
      for j in range(len(buffData)):
          # pprint.pprint(buffData[j])
          buffId = buffData[j]["buffId"]
          cond_level = buffData[j]["cond"]["level"]
          cond_phase = buffData[j]["cond"]["phase"]
          cond = f"昇進{cond_phase} Lv{cond_level}"

          buffName = building_data["buffs"][buffId]["buffName"]
          description = building_data["buffs"][buffId]["description"]
          roomType = building_data["buffs"][buffId]["roomType"]

          description = half2full(description)
          l.append([buffName, cond, room_dict[roomType], description])

  df = pd.DataFrame(l, columns=["スキル名", "習得条件", "効果対象", "効果"])

  html_table = df.to_html(index=False)
  html_table = html_table.replace("<table border=\"1\" class=\"dataframe\">", "<table class=\"uk-table\">")

  html_table = re.sub("""<thead>
    <tr style="text-align: right;">
      <th>スキル名</th>
      <th>習得条件</th>
      <th>効果対象</th>
      <th>効果</th>
    </tr>
  </thead>""",
      """<thead>
    <tr>
      <th scope="row">スキル名</th>
      <th scope="col">習得条件</th>
      <th scope="col">効果対象</th>
      <th scope="col">効果</th>
    </tr>
  </thead>""",html_table)

  html_table = re.sub("""<tr>
      <td>(.*?)</td>
      <td>(.*?)</td>
      <td>(.*?)</td>
      <td>(.*?)</td>""",
      """<tr>
      <th scope="row">\\1</th>
      <td>\\2</td>
      <td>\\3</td>
      <td>\\4</td>""",
        html_table)


  html_table = re.sub("&lt;@cc.vup&gt;(.*?)&lt;/&gt;",
        "<span class=\"tc_yellow\">\\1</span>",
        html_table)

  html_table = re.sub("&lt;@cc.kw&gt;(.*?)&lt;/&gt;",
        "<span class=\"tc_yellow\">\\1</span>",
        html_table)

  html_table = re.sub("&lt;\$cc.bd_B?&gt;&lt;@cc.rem&gt;(.*?)&lt;/&gt;&lt;/&gt;",
        "<span class=\"tc_orange\"><span class=\"uk-text-bold\"><u>\\1</u></span></span>",
        html_table)

  html_table = re.sub("&lt;\$cc.bd_a1?&gt;&lt;@cc.rem&gt;(.*?)&lt;/&gt;&lt;/&gt;",
        "<span class=\"tc_orange\"><span class=\"uk-text-bold\"><u>\\1</u></span></span>",
        html_table)

  output = "<div class=\"uk-overflow-container\">\n" + html_table + "\n</div>\n"

  return output

## モジュール

In [ ]:
def getNewModule():
  return getModuleFromUnixTime(max(list(d["timeStamp"] for d in uniequip_table["equipTrackDict"])))

def getModuleFromUnixTime(unixtime):
  data = list(d["trackList"] for d in uniequip_table["equipTrackDict"] if d["timeStamp"]==unixtime)
  for time in data:
    for track in time:
      print(track["equipId"])
  return data[0]

In [ ]:
def getdefwiki3(new,old):
  d=HtmlDiff()
  output=d.make_file(old,new)
  output=re.sub('</tbody>[\s\S]*','',re.sub('[\s\S]*<tbody>\n','',output))
  output=re.sub('</td></tr>\n','',re.sub('.*<td nowrap="nowrap">','',output))
  output=re.sub('</span><span class="diff_add">','',output)
  output=re.sub('([0-9+-\.]?)<span class="diff_add">([0-9\.])','<span class="diff_add">\\1\\2',output)
  output=re.sub('</span>([%％]?)(\.?)([0-9]*)([%％]?)','\\1\\2\\3\\4</span>',output)
  output=re.sub('</span><span class="diff_add">','',output)
  return re.sub('<span class="diff_add">','<span class="tc_yellow">',output)

In [ ]:
def modupgradeSt1(new,char_id):
  return getdefwiki3(new,character_table[char_id]["description"])
def modupgradeSt2(num,new,char_id):
  if(num==-1):
    return '<span class="tc_yellow">'+description+'</span>'
  data=[[[],[["",""],["",""],["",""]]],[[],[["",""],["",""],["",""]]]]
  for i in range(len(character_table[char_id]["talents"])):
    if(character_table[char_id]["talents"][i]["candidates"]==None):
      continue
    for j in range(len(character_table[char_id]["talents"][i]["candidates"])):
        name = character_table[char_id]["talents"][i]["candidates"][j]["name"]
        unlockCondition_pase = str(character_table[char_id]["talents"][i]["candidates"][j]["unlockCondition"]["phase"])
        unlockCondition_pase = int(unlockCondition_pase[-1:])
        unlockCondition_lv = character_table[char_id]["talents"][i]["candidates"][j]["unlockCondition"]["level"]
        requiredPotentialRank = character_table[char_id]["talents"][i]["candidates"][j]["requiredPotentialRank"]+1
        description = character_table[char_id]["talents"][i]["candidates"][j]["description"]
        if(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='1+'):
          daiX=0
        elif(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='2+'):
          daiX=1
        elif(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"]=='#'):
          daix=0
        else:
          daiX=int(character_table[char_id]["talents"][i]["candidates"][j]["prefabKey"])-1
        if(data[daiX][1][unlockCondition_pase][0]==''):
            data[daiX][1][unlockCondition_pase] = ["初期", description ]
        else:
          data[daiX][1][unlockCondition_pase][1] = senzai(data[daiX][1][unlockCondition_pase][1],description,requiredPotentialRank)
        if(name not in data[daiX][0]):
          data[daiX][0].append(name)

  for i in reversed(range(3)):
    if(data[num][1][i][1]==''):
      continue
    else:
      return getdefwiki3(new,data[num][1][i][1])
  return new

def modupgradeSt3(new,old):
  return getdefwiki3(new,old)

In [ ]:
def gen_mod_table(d):
    mod_1l = ""
    mod_1l += d[0][0]
    try:
        for k in d[0][1]:
            mod_1l += d[0][1][k] + "<p>&nbsp;</p>"
    except:
        pass

    mod_1r = ""
    try:
        mod_1r += d[0][7]
    except:
        pass
    try:
        mod_1r += d[0][5]
    except:
        pass
    try:
        mod_1r += d[0][4]
    except:
        pass
    try:
        mod_1r += d[0][3]
    except:
        pass
    try:
        mod_1r += d[0][2]
    except:
        pass

    mod_2l = ""
    mod_2l += d[1][0]
    for k in d[1][1]:
        mod_2l += f"{d[1][1][k]}"

    mod_2r = ""
    mod_2r_name = ""
    mod_2r_pre = ""
    try:
        mod_2r += d[1][8]["t"]
        mod_2r_name = d[1][8]["name"]
        if d[1][8]["talentIndex"] == -1:
            mod_2r_pre = "第二素質開放"
        if d[1][8]["talentIndex"] == 0:
            mod_2r_pre = "第一素質強化"
        if d[1][8]["talentIndex"] == 1:
            mod_2r_pre = "第二素質強化"
    except:
        pass
    try:
        mod_2r += d[1][6]["t"]
        mod_2r_name = d[1][6]["name"]
        if d[1][6]["talentIndex"] == -1:
            mod_2r_pre = "第二素質開放"
        if d[1][6]["talentIndex"] == 0:
            mod_2r_pre = "第一素質強化"
        if d[1][6]["talentIndex"] == 1:
            mod_2r_pre = "第二素質強化"
    except:
        pass

    mod_3l = ""
    mod_3l += d[2][0]
    for k in d[1][1]:
        mod_3l += f"{d[2][1][k]}"

    mod_3r = ""
    mod_3r_name = ""
    mod_3r_pre = ""
    try:
        mod_3r += d[2][8]["t"]
        mod_3r_name = d[2][8]["name"]
        if d[2][8]["talentIndex"] == -1:
            mod_3r_pre = "第二素質強化"
        if d[2][8]["talentIndex"] == 0:
            mod_3r_pre = "第一素質強化"
        if d[2][8]["talentIndex"] == 1:
            mod_3r_pre = "第二素質強化"
    except:
        pass
    try:
        mod_3r += d[2][6]["t"]
        mod_3r_name = d[2][6]["name"]
        if d[2][6]["talentIndex"] == -1:
            mod_3r_pre = "第二素質強化"
        if d[2][6]["talentIndex"] == 0:
            mod_3r_pre = "第一素質強化"
        if d[2][6]["talentIndex"] == 1:
            mod_3r_pre = "第二素質強化"
    except:
        pass

    s = f'''<div class="uk-overflow-container">
<table class="uk-table">
<tbody>
<tr>
<th><img src="https://img.wiki3.jp/arknightsjp/モジュールタイプ_{d['typeIcon'].upper()}.png">{d['typeIcon'].upper()}</th>
<th>{d["uniEquipName"]}</th>
<th colspan="1" rowspan="2"><img src="XXXXXX"></th>
</tr>
<tr>
<th>開放条件</th>
<th>
<p>昇進{d["unlockEvolvePhase"]}レベル{d["unlockLevel"]}</p>
<p>信頼度{d["unlockFavorPoint"] // 100}</p>
<p>開放任務の完了</p>
</th>
</tr>
<tr>
<th>Lv1</th>
<td>
<p>特性更新</p>
{mod_1l}
</td>
<td>
<p><span class="uk-text-bold">[特性]</span></p>
{mod_1r}
</td>
</tr>
<tr>
<th>Lv2</th>
<td>
<p>{mod_2r_pre[2:]}</p>
{mod_2l}
</td>
<td colspan="1" scope="row">
<p><span class="uk-text-bold">[{mod_2r_pre[:-2]}：{mod_2r_name}]</span></p>
<p>{mod_2r}</p>
</td>
</tr>
<tr>
<th>Lv3</th>
<td>
<p>{mod_3r_pre[2:]}</p>
{mod_3l}
</td>
<td>
<p><span class="uk-text-bold">[{mod_3r_pre[:-2]}：{mod_3r_name}]</span></p>
<p>{mod_3r}</p>
</td>
</tr>
<tr>
<th colspan="3">開放任務</th>
</tr>
<tr>
<td colspan="3">
<ul>
<li>{d["missionList"][0]}</li>
<li>{d["missionList"][1]}</li>
</ul>
</td>
</tr>
</tbody>
</table>
</div>'''

    return s

In [ ]:
blackboard_key2ja = {
    "atk": "攻撃力",
    "attack_speed": "攻撃速度",
    "block_cnt": "ブロック数",
    "cost": "コスト",
    "def": "防御力",
    "magic_resistance": "術耐性",
    "max_hp": "最大HP",
    "respawn_time": "再配置時間",
    "max_deploy_count": "max_deploy_count",
    "max_deck_stack_cnt": "max_deck_stack_cnt"
}

required_uniequip_table_equipDict = [
    "itemCost", "missionList", "typeIcon","uniEquipName",
    "unlockEvolvePhase", "unlockFavorPoint", "unlockLevel"
]

def sub4wiki3(s):
    if s == None:
        return ""
    s = re.sub("<@ba.talpu>(.*?)</>", "<span class=\"tc_cyan\">\\1</span>", s)
    s = re.sub("<@ba.kw>(.*?)</>", "<span class=\"tc_yellow\">\\1</span>", s)
    s = re.sub("<@ba.vup>(.*?)</>", "<span class=\"tc_yellow\">\\1</span>", s)
    s = re.sub("<@ba.vdown>(.*?)</>", "<span class=\"tc_coral\">\\1</span>", s)
    s = re.sub("<\$ba.(.*?)>(.*?)</>", "<span class=\"uk-text-bold\"><u>\\2</u></span>", s)
    return s

def sub4v(s, d):
    if s == None:
        return ""
    for di in d:
        for k in re.findall("{.+?}", s):
            k = k[1:-1].split(":")
            if len(k) == 1:
                if k[0] == di["key"]:
                    s = s.replace("{" + di["key"] + "}", str(int(di["value"])))
                elif k[0] == f"-{di['key']}":
                    s = s.replace("-{" + k[0] + "}", str(int(di["value"])))
            else:
                s = s.replace("{" + di["key"] + ":0%}", str(round(di["value"]*100)) + "%")
                s = s.replace("{" + di["key"] + ":0}", str(int(di["value"])))
    return s
def generate_module_dict(char_id: str):
    ret = []
    st2=''
    st2f=True
    if not char_id.startswith("char"):
        # print(f"char_id [{char_id}] is not operator.")
        return 0

    if not char_id in uniequip_table["charEquip"]:
        # print(f"char_id [{char_id}] has no module.")
        return 0

    for mod_id in uniequip_table["charEquip"][char_id]:
        if mod_id.startswith("uniequip_001"):
            continue

        mod_info = {}
        d = {}
        for k, v in uniequip_table["equipDict"][mod_id].items():
            if k in required_uniequip_table_equipDict:
                d[k] = v

        mod_info["mod_id"] = mod_id
        mod_info["typeIcon"] = d["typeIcon"]
        mod_info["uniEquipName"] = d["uniEquipName"]
        mod_info["unlockEvolvePhase"] = d["unlockEvolvePhase"]
        mod_info["unlockLevel"] = d["unlockLevel"]
        mod_info["unlockFavorPoint"] = d["unlockFavorPoint"]

        mod_info["itemCost"] = d["itemCost"]

        mod_info["missionList"] = []
        for num, mission in enumerate(d["missionList"]):
             mod_info["missionList"].append(uniequip_table["missionList"][mission]["desc"])

        for cnt, phases in enumerate(battle_equip_table[mod_id]["phases"]):
            mod_info[cnt] = {}
            for phase in phases:
                # if phase == "equipLevel":
                #     print(f"lv.{phases[phase]}", "===============================")

                if phase == "attributeBlackboard":
                    status_up = ""
                    for ab in phases[phase]:
                        t = "<p>" + blackboard_key2ja[ab["key"]] + "<span class=\"tc_yellow\">"
                        if ab["value"] > 0:
                            t += f"+{int(ab['value'])}"
                        else:
                            t += f"{int(ab['value'])}"
                        if ab['key'] == "respawn_time":
                            t += "秒"
                        t = half2full(t)
                        t += "</span></p>"
                        status_up += t
                    mod_info[cnt][0] = status_up

                if phase == "tokenAttributeBlackboard":
                    mod_info[cnt][1] = {}
                    for tab_k in phases[phase]:
                        status_up = f"<p>{tab_k}</p>"
                        for ab in phases[phase][tab_k]:
                            t = "<p>" + blackboard_key2ja[ab["key"]] + "<span class=\"tc_yellow\">"
                            if ab["value"] > 0:
                                t += f"+{int(ab['value'])}"
                            else:
                                t += f"{int(ab['value'])}"
                            if ab['key'] == "respawn_time":
                                t += "秒"
                            t = half2full(t)
                            t += "</span></p>"
                            status_up += t
                        mod_info[cnt][1][tab_k] = status_up

                if phase == "parts":
                    for part in phases[phase]:
                        if part["target"] == "DISPLAY":
                            blackboard = part["overrideTraitDataBundle"]["candidates"][0]["blackboard"]
                            t = sub4v(part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"], blackboard)
                            t = '<span class="tc_yellow">＋</span>'+t
                            t = half2full(t)
                            t = sub4wiki3(t)
                            t = f"<p>{t}</p>"
                            if "攻撃範囲" in t:
                                t += f"<p><img src=\"https://img.wiki3.jp/arknightsjp/{mod_info['typeIcon'].upper()}_range.jpg\"></p>"
                            mod_info[cnt][2] = t

                            if not part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"] == None:
                                t = sub4wiki3(part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"])
                                t = f"<p>{t}</p>"
                                mod_info[cnt][3] = t

                        if part["target"] == "TRAIT":
                            if not part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"] == None:
                                blackboard = part["overrideTraitDataBundle"]["candidates"][0]["blackboard"]
                                t = sub4v(part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"], blackboard)
                                t = '<span class="tc_yellow">＋</span>'+t
                                t = half2full(t)
                                t = sub4wiki3(t)
                                t = f"<p>{t}</p>"
                                mod_info[cnt][4] = t

                            if not part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"] == None:
                                t = sub4wiki3(part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"])
                                t = modupgradeSt1(t,char_id)
                                t = f"<p>{t}</p>"
                                mod_info[cnt][5] = t

                        if part["target"] == "TALENT":
                            if not part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"] == None:
                                name = part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"]
                                talentIndex = part["addOrOverrideTalentDataBundle"]["candidates"][0]["talentIndex"]
                                t = part["addOrOverrideTalentDataBundle"]["candidates"][0]["upgradeDescription"]
                                if(st2f):
                                  st2 = t
                                  st2f=False
                                  t = modupgradeSt2(talentIndex,t,char_id)
                                else:
                                  t= modupgradeSt3(t,st2)
                                t = half2full(t)
                                t = sub4wiki3(t)
                                mod_info[cnt][6] = {"talentIndex": talentIndex, "name": name, "t": t}

                        if part["target"] == "TRAIT_DATA_ONLY":
                            for Trait in part["overrideTraitDataBundle"]["candidates"]:
                                blackboard = Trait["blackboard"]
                                t = sub4v(Trait["overrideDescripton"], blackboard)
                                t = modupgradeSt1(t,char_id)
                                t = sub4wiki3(t)
                                t = f"<p>{t}</p>"
                                mod_info[cnt][7] = t

                        if part["target"] == "TALENT_DATA_ONLY":
                            if not part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"] == None:
                                name = part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"]
                                talentIndex = part["addOrOverrideTalentDataBundle"]["candidates"][0]["talentIndex"]
                                s1 = part["addOrOverrideTalentDataBundle"]["candidates"][0]["upgradeDescription"]
                                t = s1
                                try:
                                    s2 = part["addOrOverrideTalentDataBundle"]["candidates"][1]["upgradeDescription"]
                                    pr = part["addOrOverrideTalentDataBundle"]["candidates"][1]['requiredPotentialRank'] + 1
                                    t = senzai(s1, s2, pr)
                                except:
                                    pass
                                if(st2f):
                                  st2 = t
                                  st2f=False
                                  t = modupgradeSt2(talentIndex,t,char_id)
                                else:
                                  t= modupgradeSt3(t,st2)
                                t = half2full(t)
                                t = sub4wiki3(t)
                                mod_info[cnt][8] = {"talentIndex": talentIndex, "name": name, "t": t}

        ret.append(mod_info)
        # pprint.pprint(mod_info)
    return ret


In [ ]:
def generate_module_dict_from_mod_id(mod_id: str,char_id:str):
    ret = []
    if mod_id.startswith("uniequip_001"):
        return 0
    st2=''
    st2f=True
    mod_info = {}
    d = {}
    for k, v in uniequip_table["equipDict"][mod_id].items():
        if k in required_uniequip_table_equipDict:
            d[k] = v

    mod_info["mod_id"] = mod_id
    mod_info["typeIcon"] = d["typeIcon"]
    mod_info["uniEquipName"] = d["uniEquipName"]
    mod_info["unlockEvolvePhase"] = d["unlockEvolvePhase"]
    mod_info["unlockLevel"] = d["unlockLevel"]
    mod_info["unlockFavorPoint"] = d["unlockFavorPoint"]

    mod_info["itemCost"] = d["itemCost"]


    mod_info["missionList"] = []
    for num, mission in enumerate(d["missionList"]):
            mod_info["missionList"].append(uniequip_table["missionList"][mission]["desc"])

    for cnt, phases in enumerate(battle_equip_table[mod_id]["phases"]):
        mod_info[cnt] = {}
        for phase in phases:
            # if phase == "equipLevel":
            #     print(f"lv.{phases[phase]}", "===============================")

            if phase == "attributeBlackboard":
                status_up = ""
                for ab in phases[phase]:
                    t = "<p>" + blackboard_key2ja[ab["key"]] + "<span class=\"tc_yellow\">"
                    if ab["value"] > 0:
                        t += f"+{int(ab['value'])}"
                    else:
                        t += f"{int(ab['value'])}"
                    if ab['key'] == "respawn_time":
                        t += "秒"
                    t = half2full(t)
                    t += "</span></p>"
                    status_up += t
                mod_info[cnt][0] = status_up

            if phase == "tokenAttributeBlackboard":
                mod_info[cnt][1] = {}
                for tab_k in phases[phase]:
                    status_up = f"<p>{tab_k}</p>"
                    for ab in phases[phase][tab_k]:
                        t = "<p>" + blackboard_key2ja[ab["key"]] + "<span class=\"tc_yellow\">"
                        if ab["value"] > 0:
                            t += f"+{int(ab['value'])}"
                        else:
                            t += f"{int(ab['value'])}"
                        if ab['key'] == "respawn_time":
                            t += "秒"
                        t = half2full(t)
                        t += "</span></p>"
                        status_up += t
                    mod_info[cnt][1][tab_k] = status_up

            if phase == "parts":
                for part in phases[phase]:
                    if part["target"] == "DISPLAY":
                        blackboard = part["overrideTraitDataBundle"]["candidates"][0]["blackboard"]
                        t = sub4v(part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"], blackboard)
                        t = '<span class="tc_yellow">＋</span>'+t
                        t = half2full(t)
                        t = sub4wiki3(t)
                        t = f"<p>{t}</p>"
                        if "攻撃範囲" in t:
                            t += f"<p><img src=\"https://img.wiki3.jp/arknightsjp/{mod_info['typeIcon'].upper()}_range.jpg\"></p>"
                        mod_info[cnt][2] = t

                        if not part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"] == None:
                            t = sub4wiki3(part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"])
                            t = f"<p>{t}</p>"
                            mod_info[cnt][3] = t

                    if part["target"] == "TRAIT":
                        if not part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"] == None:
                            blackboard = part["overrideTraitDataBundle"]["candidates"][0]["blackboard"]
                            t = sub4v(part["overrideTraitDataBundle"]["candidates"][0]["additionalDescription"], blackboard)
                            t = '<span class="tc_yellow">＋</span>'+t
                            t = half2full(t)
                            t = sub4wiki3(t)
                            t = f"<p>{t}</p>"
                            mod_info[cnt][4] = t

                        if not part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"] == None:
                            t = sub4wiki3(part["overrideTraitDataBundle"]["candidates"][0]["overrideDescripton"])
                            t = modupgradeSt1(t,char_id)
                            t = f"<p>{t}</p>"
                            mod_info[cnt][5] = t

                    if part["target"] == "TALENT":
                        if not part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"] == None:
                            name = part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"]
                            talentIndex = part["addOrOverrideTalentDataBundle"]["candidates"][0]["talentIndex"]
                            t = part["addOrOverrideTalentDataBundle"]["candidates"][0]["upgradeDescription"]
                            if(st2f):
                              st2 = t
                              st2f=False
                              t = modupgradeSt2(talentIndex,t,char_id)
                            else:
                              t= modupgradeSt3(t,st2)
                            t = half2full(t)
                            t = sub4wiki3(t)
                            mod_info[cnt][6] = {"talentIndex": talentIndex, "name": name, "t": t}

                    if part["target"] == "TRAIT_DATA_ONLY":
                        for Trait in part["overrideTraitDataBundle"]["candidates"]:
                            blackboard = Trait["blackboard"]
                            t = sub4v(Trait["overrideDescripton"], blackboard)
                            t = modupgradeSt1(t,char_id)
                            t = sub4wiki3(t)
                            t = f"<p>{t}</p>"
                            mod_info[cnt][7] = t

                    if part["target"] == "TALENT_DATA_ONLY":
                        if not part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"] == None:
                            name = part["addOrOverrideTalentDataBundle"]["candidates"][0]["name"]
                            talentIndex = part["addOrOverrideTalentDataBundle"]["candidates"][0]["talentIndex"]
                            s1 = part["addOrOverrideTalentDataBundle"]["candidates"][0]["upgradeDescription"]
                            t = s1
                            try:
                                s2 = part["addOrOverrideTalentDataBundle"]["candidates"][1]["upgradeDescription"]
                                pr = part["addOrOverrideTalentDataBundle"]["candidates"][1]['requiredPotentialRank'] + 1
                                t = senzai(s1, s2, pr)
                            except:
                                pass
                            if(st2f):
                              st2 = t
                              st2f=False
                              t = modupgradeSt2(talentIndex,t,char_id)
                            else:
                              t= modupgradeSt3(t,st2)
                            t = half2full(t)
                            t = sub4wiki3(t)
                            mod_info[cnt][8] = {"talentIndex": talentIndex, "name": name, "t": t}

    ret.append(mod_info)
    # pprint.pprint(mod_info)
    return ret

In [ ]:
def gen_mod_levelup_items_table(d):
    ret = [
        '<table class="uk-table">',
        '<thead>',
        '<tr>',
        '<th scope="row">レベル</th>',
        '<th scope="col">必要素材</th>',
        '</tr>',
        '</thead>',
        '<tbody>'
    ]
    typeIcon = d["typeIcon"].upper()
    typeIcon_url = f"<img src=\"https://img.wiki3.jp/arknightsjp/モジュールタイプ_{typeIcon}.png\">"
    uniEquipName = d["uniEquipName"]
    for level, itemlist in d["itemCost"].items():
        ret.extend([
            '<tr>',
            '<th scope="row">',
            f'<p>{typeIcon_url}{typeIcon}</p>',
            f'<p>Lv{level}</p></th>',
            '<td>'
        ])
        for item in itemlist:
            item_name = getItemName(item["id"])
            item_count = item["count"]
            ret.extend([
                f'<p>{getWiki3Contents(item_name)}&nbsp;x{item_count}</p>'
            ])
        ret.extend(['</td>','</tr>'])
    ret.extend(['</tbody>','</table>'])

    ret = "\n".join(ret)
    return ret

# 実行用

## キャラID指定式単一実行用

In [ ]:
print(add)

['char_341_sntlla', 'char_2012_typhon']


In [ ]:
print(charTable("char_341_sntlla"))

<h1>寒檀</h1>
<h3 id="content_header_main_content_h3_5">素質</h3>
<p>nationId：サーミ</p><p>groupId：None</p><p>teamId：None</p><p>募集タグ：【群攻】【控场】</p><p>コスト：30&nbsp;;&nbsp;33&nbsp;;&nbsp;33&nbsp;;&nbsp;</p><p>再配置時間：70s</p><p>&nbsp;</p><div class="uk-overflow-container"><table class="uk-table"><thead><tr><th scope="row">&nbsp;</th><th scope="col">初期</th><th scope="col">初期MAX</th><th scope="col">昇進1MAX</th><th scope="col">昇進2MAX</th><th scope="col">信頼度</th></tr></thead><tbody><tr><th>最大HP</th><td class="uk-text-right">736</td><td class="uk-text-right">1023</td><td class="uk-text-right">1279</td><td class="uk-text-right">1640</td><td class=></td></tr><tr><th>攻撃力</th><td class="uk-text-right">357</td><td class="uk-text-right">503</td><td class="uk-text-right">646</td><td class="uk-text-right">770</td><td class=>＋90</td></tr><tr><th>防御力</th><td class="uk-text-right">49</td><td class="uk-text-right">75</td><td class="uk-text-right">103</td><td class="uk-text-right">123</td><td class=></td></tr><tr><th>術

## 新キャラ一括用

In [ ]:
text=""
for char_id in add:
  print(char_id)
  text+=charTable(char_id)

p = Path('/content/drive/MyDrive/ArknightsGameData_autoWiki3/'+
         datetime.datetime.now().astimezone(timezone('Asia/Tokyo')).strftime('%m%d_%H%M')+'_Character.txt')
print(p.write_text(text))

char_341_sntlla


IndexError: ignored

## 新モジュール一括用

In [ ]:
text='<h1>モジュール</h1>'
end=[]
targetlist=getNewModule()
for mod in targetlist:
  ret=generate_module_dict_from_mod_id(mod['equipId'],mod['charId'])
  if ret == 0 :
    continue
  for d in ret:
    if(d in end):
      continue
    text+=gen_mod_table(d)+"<p>&nbsp;</p>"
    text+=gen_mod_levelup_items_table(d)+"<p>&nbsp;</p>"
    end.append(d)
p = Path('/content/drive/MyDrive/ArknightsGameData_autoWiki3/'+
         datetime.datetime.now().astimezone(timezone('Asia/Tokyo')).strftime('%m%d_%H%M')+'_Module.txt')
print(p.write_text(text))

uniequip_001_sntlla
uniequip_002_sntlla
uniequip_001_elysm
uniequip_002_elysm
uniequip_001_sleach
uniequip_002_sleach
uniequip_001_myrtle
uniequip_002_myrtle
uniequip_001_typhon
uniequip_002_typhon
uniequip_003_helage
uniequip_003_phatom
24961
